In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import time
import random
import time
import requests 
import re
import logging
import subprocess
from subprocess import Popen
from sys import platform
import os, sys
import logging
import json
import threading

from netunicorn.client.remote import RemoteClient, RemoteClientException
from netunicorn.base import Experiment, ExperimentStatus, Pipeline
from netunicorn.library.tasks.basic import SleepTask
from netunicorn.library.tasks.measurements.ping import Ping
from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.base.task import Failure, Task, TaskDispatcher
from netunicorn.base import Result, Failure, Success, Task, TaskDispatcher
from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node

# our imports
#watchers
from netunicorn.library.tasks.video_watchers.youtube_watcher import WatchYouTubeVideo
from netunicorn.library.tasks.video_watchers.vimeo_watcher import WatchVimeoVideo
# add vimeo and twitch
#pcaps
from netunicorn.library.tasks.capture.tcpdump import StartCapture, StopNamedCapture
#speedtest
from netunicorn.library.tasks.measurements.ookla_speedtest import SpeedTest

from netunicorn.library.tasks.upload.fileio import UploadToFileIO

from typing import Dict
from typing import Optional
from enum import IntEnum
from datetime import datetime

from typing import Dict
from typing import Optional
from enum import IntEnum
from datetime import datetime

from returns.pipeline import is_successful
from returns.result import Failure

In [3]:
NETUNICORN_ENDPOINT = os.environ.get('NETUNICORN_ENDPOINT', 'https://pinot.cs.ucsb.edu/netunicorn')
NETUNICORN_LOGIN = os.environ.get('NETUNICORN_LOGIN', 'team_tz')       
NETUNICORN_PASSWORD = os.environ.get('NETUNICORN_PASSWORD', 'throughput445')

In [5]:
client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
print("Health Check: {}".format(client.healthcheck()))
nodes = client.get_nodes()
print(nodes)
print(len(nodes))
raspi_nodes = nodes.filter(lambda node: node.name.startswith("raspi")).take(10)
print(raspi_nodes)

Health Check: True
[<Uncountable node pool with next node template: [aws-fargate-A-, aws-fargate-B-, aws-fargate-ARM64-]>, [snl-server-5, atopnuc-84:47:09:1b:00:3c, atopnuc-84:47:09:17:c0:b6, atopnuc-84:47:09:17:c1:b7, atopnuc-84:47:09:16:b6:cf, atopnuc-84:47:09:17:c0:57, atopnuc-84:47:09:17:c1:df, atopnuc-84:47:09:17:c8:0c, atopnuc-84:47:09:17:c4:83, atopnuc-84:47:09:17:c0:f6, atopnuc-84:47:09:17:c2:14, atopnuc-84:47:09:17:c1:8d, raspi-e4:5f:01:72:a2:35, raspi-e4:5f:01:8d:f5:95, raspi-e4:5f:01:6f:ef:0a, raspi-e4:5f:01:56:d8:cd, raspi-e4:5f:01:56:d8:f3, raspi-e4:5f:01:56:d9:8b, raspi-e4:5f:01:9c:ca:3a, raspi-e4:5f:01:9c:20:81, raspi-e4:5f:01:75:6b:2c, raspi-e4:5f:01:56:d9:a2, raspi-e4:5f:01:8d:ca:31, raspi-e4:5f:01:ac:e5:4b, raspi-e4:5f:01:a0:4f:96, raspi-e4:5f:01:72:a2:eb, raspi-e4:5f:01:72:a3:1c, raspi-e4:5f:01:79:4a:18, raspi-e4:5f:01:96:01:e8, raspi-e4:5f:01:75:54:ec, raspi-e4:5f:01:ad:c9:3f, raspi-e4:5f:01:75:ae:8d, raspi-e4:5f:01:56:d9:0a, raspi-e4:5f:01:a7:ae:70, raspi-e4:5f:01:

In [9]:
pipeline = (Pipeline()
                .then(SpeedTest())
                .then(StartCapture(filepath='/tmp/videocapture.pcap', name="capture"))
                .then(SleepTask(seconds=2))
                .then(WatchVimeoVideo(video_url="https://vimeo.com/874415317#t=44s", duration=10))
                .then(SleepTask(seconds=2))
                .then(StopNamedCapture(start_capture_task_name="capture"))
                .then(UploadToFileIO(filepath='/tmp/videocapture.pcap', expires="1d"))
                .then(SpeedTest())
)
                    

In [10]:
def execute_pipeline(pipeline, working_node, experiment_label):
    client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
    nodes = client.get_nodes()
    working_nodes = nodes.filter(lambda node: node.name.startswith(working_node)).take(1)
    experiment = Experiment().map(pipeline, working_nodes)

    from netunicorn.base import DockerImage
    for deployment in experiment:
        # you can explore the image on the DockerHub
        deployment.environment_definition = DockerImage(image='speeeday/chromium-speedtest:0.3.0')

    try:
        client.delete_experiment(experiment_label)
    except RemoteClientException:
        pass

    # Prepare Experiment
    client.prepare_experiment(experiment, experiment_label)
    while True:
        info = client.get_experiment_status(experiment_label)
        print(info.status)
        if info.status == ExperimentStatus.READY:
            break
        time.sleep(20)

    time.sleep(5)

    # Execute Experiment
    client.start_execution(experiment_label)
    while True:
        info = client.get_experiment_status(experiment_label)
        print(info.status)
        if info.status != ExperimentStatus.RUNNING:
            break
        time.sleep(20)
    return info

In [11]:
def display_results(info):
    # Get Results
    for report in info.execution_result:
        print(f"Node name: {report.node.name}")
        print(f"Error: {report.error}")

        if report.result is None:
            print("report.result is EMPTY..")
            continue

        result, log = report.result  # report stores results of execution and corresponding log

        # result is a returns.result.Result object, could be Success of Failure
        print(f"Result is: {type(result)}")
        if is_successful(result):
            data = result.unwrap()
        else:
            data = result.failure()
        try:
            for key, value in data.items():
                print(f"{key}: {value}")
        except:
            print(f"No attribute 'items' in result")

        # we also can explore logs
        for line in log:
            print(line.strip())
        print()

In [12]:
working_node = 'raspi-e4:5f:01:9b:85:9c'

In [13]:
info = execute_pipeline(pipeline, working_node, 'vimeo_experiment2')
display_results(info)

ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.READY
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.FINISHED
Node name: raspi-e4:5f:01:9b:85:9c
Error: None
Result is: <class 'returns.result.Success'>
container_value: defaultdict(<class 'list'>, {'e6244ae5-3769-4cb5-9a4c-a897628934c3': [<Success: {'container_value': {'ping': {'value': 8.134, 'unit': 'ms'}, 'download': {'value': 66.94, 'unit': 'Mbit/s'}, 'upload': {'value': 60.62, 'unit': 'Mbit/s'}}}>], 'capture': [<Success: {'container_value': 115}>], '0e04a24c-06a6-43dc-89a4-afab076f1b8c': [<Success: {'container_value': 2}>], 'a1b1130f-e0e9-49d3-9503-f8afeeda2ab8': [<Success: {'container_value': 'Video finished by timeout: 20 seconds'}>], 'a3a6ed05-865d-4ceb-83c5-

In [14]:
display_results(info)

Node name: raspi-e4:5f:01:9b:85:9c
Error: None
Result is: <class 'returns.result.Success'>
container_value: defaultdict(<class 'list'>, {'e6244ae5-3769-4cb5-9a4c-a897628934c3': [<Success: {'container_value': {'ping': {'value': 8.134, 'unit': 'ms'}, 'download': {'value': 66.94, 'unit': 'Mbit/s'}, 'upload': {'value': 60.62, 'unit': 'Mbit/s'}}}>], 'capture': [<Success: {'container_value': 115}>], '0e04a24c-06a6-43dc-89a4-afab076f1b8c': [<Success: {'container_value': 2}>], 'a1b1130f-e0e9-49d3-9503-f8afeeda2ab8': [<Success: {'container_value': 'Video finished by timeout: 20 seconds'}>], 'a3a6ed05-865d-4ceb-83c5-9280cd526f93': [<Success: {'container_value': 2}>], '973cd918-3e62-4b16-bf78-58ff509a9f8b': [<Success: {'container_value': b''}>], '246dc54b-a538-4cf2-ada7-389d953589e6': [<Success: {'container_value': '{"success":true,"status":200,"id":"60dd6740-8130-11ee-a404-7584938b3e2d","key":"Qmpb5kM0dICP","path":"/","nodeType":"file","name":"videocapture.pcap","title":null,"description":null,"